# Strait of Georgia (SOCGS) HF Radar Surface Currents

![gif](../figures/sogcs_hf_radar.gif "gif")

In [1]:
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt

from datetime import datetime, timezone, timedelta
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import pandas as pd
import os
import xarray as xr

from ONCToolbox import ONCToolbox, format_datetime
from ONCToolbox.utils.locations import HFSites

In [2]:
save_dir = 'onc_data'
onct = ONCToolbox(save_dir=save_dir)

In [3]:
loc_code = 'SOGCS'
dev_cat_code = 'OCEANOGRAPHICRADAR'
data_prod_code = 'CODARQCSC'
ext = 'nc'

date_to = datetime.now(timezone.utc) # End of the data request.
date_from = date_to - timedelta(days = 10) # Start of the data request.

params = {'locationCode': loc_code,
          'deviceCategoryCode': dev_cat_code,
          'dateFrom': format_datetime(date_from),
          'dateTo': format_datetime(date_to),
          'extension': ext,
          'dataProductCode': data_prod_code}

req = onct.requestDataProduct(filters = params)
status = onct.runDataProduct(req['dpRequestId'])
data_files = []
meta_files = []
for run_id in status['runIds']:
    down = onct.downloadDataProduct(run_id)
    for file_info in down:
        fn = file_info['file']
        fp = os.path.join(save_dir,fn)
        if fn.endswith('.nc'):
            data_files.append(fp)
        elif fn.endswith('.xml'):
            meta_files.append(fp)

Request Id: 27210916
Estimated File Size: 8 MB
Estimated Processing Time: 15 min
To cancel the running data product, run 'onc.cancelDataProduct(27210916)'

   queued
   data product running.................................................................
   1 files generated for this data product
   complete

   Skipping "StraitofGeorgia_StraitofGeorgiaCODARSystem_OceanographicRadarSystem_20250906T190000.000Z_20250916T170000.000Z-Totals_Clean.nc": File already exists.
   Skipping "StraitofGeorgia_StraitofGeorgiaCODARSystem_OceanographicRadarSystem_20250906T190000Z_20250916T170000Z_NC_META.xml": File already exists.


In [7]:
ds_list = [xr.open_dataset(f) for f in data_files]
hf = xr.concat(ds_list, dim='time', combine_attrs = 'drop_conflicts')

In [8]:
hf['time'] = hf.time.dt.round('1h')
hf = hf.where(hf.quality_flag != 4, drop = True)
hf = hf[['u','v']]
hf['spd'] = np.sqrt(hf.u**2 + hf.v**2)  # Calculate speed for colormap.
hf = hf.dropna(dim = 'time', how = 'all')

In [9]:
hfcmap = matplotlib.colormaps.get_cmap('jet')
hfnorm = matplotlib.colors.Normalize(vmin = 0 , vmax = 100)

tile = cimgt.GoogleTiles(style = 'satellite')

xmin = hf.lon.min()-0.01
xmax = hf.lon.max()+0.01
ymin = hf.lat.min()-0.01
ymax = hf.lat.max()+0.01
x_major_locator = 0.3
y_major_locator = 0.2

t = hf.time.values
x = hf.lon.values
y = hf.lat.values
c = hf.spd
u = hf.u
v = hf.v

crs = ccrs.PlateCarree()
fig, ax = plt.subplots(1,1,figsize = (8,6), constrained_layout = True, subplot_kw={'projection': crs})
ax.add_image(tile,8,alpha = 0.5)
#ax.coastlines()
ax.set_extent([xmin, xmax, ymin, ymax])

ax.set_xticks(np.arange(xmin, xmax,x_major_locator))
ax.set_xlabel(r'Longitude ($^{\circ}E$)')

ax.set_yticks(np.arange(ymin, ymax,y_major_locator))
ax.set_ylabel(r'Latitude ($^{\circ}N$)')

quiv = ax.quiver(x,y,u[:,:,0],v[:,:,0],c[:,:,0], cmap = hfcmap, norm = hfnorm, transform = crs, angles = 'uv',pivot = 'mid')
fig.colorbar(quiv, ax = ax, shrink = 0.45, label = 'Surface Current Speed (cm/s)')

# hf_sites = [k for k, v in dict(HFSites.__dict__).items() if '__' not in k]
# for hf_site in hf_sites:
#     site = getattr(HFSites, hf_site)
#     ax.plot(site.lon, site.lat, marker = '^', color = 'k', markersize = 8, transform = crs)
#     ax.text(site.lon+0.01, site.lat+0.01, hf_site, fontsize = 9, transform = crs)

def update(frame):
    _u = u[:,:,frame]
    _v = v[:,:,frame]
    _c = c[:,:,frame]
    
    quiv.set_UVC(_u,_v,_c)
    
    ax.set_title(pd.to_datetime(t[frame]).strftime('%Y-%m-%dT%H:%M:%SZ'))
    
    return quiv,


ani = FuncAnimation(fig=fig, func=update, frames=len(t), interval=1000, blit = False)
ani.save('../figures/sogcs_hf_radar.gif')
plt.close()